# Database Application: International Real Estate
An international real estate company has offices around the world. 
- Real estate agents can bea ssigned to any of the offices. 
- Each house is assigned one office, one seller and one real estate agent. 
- Sellers can sell several houses. 
- Buyers can buy several houses.

### Use of data normalization, indices and transactions

**Data Normalization**
- The database has four base tables (offices, agents, sellers, buyers) and one house listings table that are created initially. 
- Three additional tables are populated as sales occur (sales, commissions).
- A summary table of agents' commission is provided in response to (3).
- **1NF**: 
    - each column has only one attribute: 'id', 'first_name', 'bathrooms', ...
    - all values in a column are of the same type: defined as  Column(Text), Column(Integer), ...
    - all columns have unique names: 'office_id', 'zipcode'
    - order in which data is stored does not matter: which office/ agent/ listing ... is entered and saved first does not matter (important to note that that does not include dependencies! houselistings that refer to a seller needs to have a seller saved first)
- **2NF**:
    - first normal form: check
    - no partial dependency: columns are only dependent on the primary key, and nothing else. For example, if the Sales table had sellers as a primary key, but sellers could have multiple houses, the entries would not distinguish themselves by the seller. The current table set up has clear and unique primary keys, where no column information depends on more than one key identifier (id)
- **3NF**:
    - second normal form: check
    - no transitive dependency: For example the House table has agent_id, buyer_id and seller_id, all of which are dependent on the house entry id. However, it does not include their names or contact detail, as those attributes are dependent on the agent_id, buyer_id and seller_id themselves. They are stored in the Agent, Buyer and Seller table respectively.


Source: https://www.studytonight.com/dbms/database-normalization.php

**Indices**
- "Every primary key (in every popular database) will be automatically indexed. This is to make sure that testing for a key’s uniqueness (when inserting a new row) is fast." (Professor Philip Sterne, Slack, March 21)
- Any joins occurring in this implementation are based on foreign key constraints, which use the id of another column, which is their already indexed primary key. Thus, for joins, other indices would not increase efficiency. 
- For queries, I implemented covering indices for the four base tables (Office, Agent, Seller, Buyer), since their additional information such as name or contact are more often looked up, and when such information is stored in a covering index, it does not need to be looked up in the original table. 
- To my understanding, no other indices would improve performance of the queries, since due to the unique primary key that already exists, the main requirements are met. If there were join tables or no unique id primary keys, composite keys could be useful (e.g. offices and agents, which have a many-to-many relationship). 


**Transactions**
- Transactions are necessary when an action (e.g. selling a house) leads to multiple edits to the database (e.g. updating the sold status and adding to the Sale database). To ensure that all necessary edits of happen in one go, they are wrapped in a transaction. Either all go through, or none go through. 
- This is implemented using the sessionmaker of SQLAlchemy. Tasks are added to the transaction using `session.add(entry)` and executed using `session.commit()`. If the transaction runs into an error, it rolls back to the previous state before the transaction was executed, and the session has to be closed with `session.close()` and retried. 
- This database uses transactions when adding the sale of a house (see function house_sale()). A single transaction (1) queries results from the database, (2) adds an entry to the Sale data and (3) adds an entry to the commission table. 
- SQLAlchmey's implementation of transactions is ACID: **(A)** tomic, as it is one single unit of a house sale, **(C)** onsistent as changes are not visible before all have been implemented but the old version of the tables are still visible, **(I)** solated as the updates occurr isolated from the database first and keep it intact and **(D)** urable as transactions, once committed, remain committed even if interruptions occur. 


### Imports

In [1]:
from sqlalchemy import create_engine, Column, Text, Integer, Date, Boolean, ForeignKey
from sqlalchemy import case, func, join, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker 
import pandas as pd
import datetime
import pandas.io.sql as psql


### Tables

In [2]:
engine = create_engine('sqlite:///database.db')
engine.connect() 

Base = declarative_base() 


#--------------------------------------------------
# Base Tables
#--------------------------------------------------

class Office(Base):
    __tablename__ = 'offices'
    id = Column(Integer, primary_key = True)
    location = Column(Text, index = True)
    
    def __repr__(self):
        return "<Office(id={0}, location={1})".format(self.id, self.location)


class Agent(Base):
    __tablename__ = 'agents'
    id = Column(Integer, primary_key = True)
    first_name = Column(Text, index = True)
    last_name = Column(Text, index = True)
    email = Column(Text, index=True)
    
    def __repr__(self):
        return "<Buyer(id={0}, first_name={1}, last_name={2})".format(self.id, self.first_name, self.last_name)

    
class Seller(Base):
    __tablename__ = 'sellers'
    id = Column(Integer, primary_key = True)
    first_name = Column(Text, index = True)
    last_name = Column(Text, index = True)
    
    def __repr__(self):
        return "<Seller(id={0}, first_name={1}, last_name={2})".format(self.id, self.first_name, self.last_name)


class Buyer(Base):
    __tablename__ = 'buyers'
    id = Column(Integer, primary_key = True)
    first_name = Column(Text, index = True)
    last_name = Column(Text, index = True)
    
    def __repr__(self):
        return "<Buyer(id={0}, first_name={1}, last_name={2})".format(self.id, self.first_name, self.last_name)


#--------------------------------------------------
# House Listings
#--------------------------------------------------


class House(Base):
    __tablename__ = 'houses'
    id = Column(Integer, primary_key = True)
    name = Column(Text)
    office_id = Column(Integer, ForeignKey('offices.id'))
    agent_id = Column(Integer, ForeignKey('agents.id'))
    seller_id = Column(Integer, ForeignKey('sellers.id'))    
    bedrooms = Column(Integer, nullable=False)
    bathrooms = Column(Integer, nullable=False)
    listing_price = Column(Integer, nullable=False)
    zipcode = Column(Integer)
    listing_date = Column(Date)
    sold = Column(Boolean)

    def __repr__(self):
        return "<House(id={0}, name={1}, sold={2})>".format(self.id, self.name, self.sold)


#--------------------------------------------------
# Sale and Commission
#--------------------------------------------------

class Sale(Base):
    __tablename__ = 'sales'
    id = Column(Integer, primary_key = True)
    house_id = Column(Integer, ForeignKey('houses.id'))
    buyer_id = Column(Integer, ForeignKey('buyers.id'))
    sale_price = Column(Integer)
    sale_date = Column(Date)
    
    def __repr__(self):
        return "<Sale(id={0}, house_id={1}, buyer_id={2})>".format(self.id, self.house_id, self.buyer_id)
    

class Commission(Base):
    __tablename__ = 'commissions'
    id = Column(Integer, primary_key = True)
    agent_id = Column(Integer, ForeignKey('agents.id'))
    sale_id = Column(Integer, ForeignKey('sales.id'))
    commission = Column(Integer)
    
    def __repr__(self):
        return "<Commission(id={0}, agent_id={1}, commission={2})>".format(self.id, self.agent_id, self.commission)


### Insert Initial Data

In [3]:
Base.metadata.create_all(bind=engine) 

Session = sessionmaker(bind=engine)
session = Session()

#--------------------------------------------------
# Create offices, agents, sellers and buyers
#--------------------------------------------------


office_keys = ['id', 'location']
office_values = [
    [1001, 'San Francisco'],
    [1002, 'Seoul'],
    [1003, 'Hyderabad'],
    [1004, 'Berlin'],
    [1005, 'Buenos Aires'],
    [1006, 'London'],
    [1007, 'Taipei']]


agent_keys = ['id', 'first_name', 'last_name', 'email']
agent_values = [
    [2001, 'Anna', 'Aruya', 'anna@google.com'],
    [2002, 'Anton', 'Aldi', 'anton@google.com'],
    [2003, 'Ashish', 'Alder', 'ashish@google.com'],
    [2004, 'Angela', 'Anger', 'angela@google.com'],
    [2005, 'Adrian', 'Aldi', 'adrian@google.com'],
    [2006, 'Ari', 'Aro', 'ari@google.com'],
    [2007, 'Anne', 'Arido', 'anne@google.com'],
    [2008, 'Alinga', 'Azon', 'alinga@google.com'],
    [2009, 'Abuli', 'Appo', 'abuli@google.com'],
    [2010, 'Ankita', 'Avush', 'ankita@google.com']]


seller_keys = ['id', 'first_name', 'last_name']
seller_values = [
    [3001, 'Sabine', 'Singer'],
    [3002, 'Sergio', 'Sent'],
    [3003, 'Sasha', 'Sorrow'],
    [3004, 'Stub', 'Stringo'],
    [3005, 'Stella', 'Serg'],
    [3006, 'Sanik', 'Soumka'],
    [3007, 'Soumaya', 'Strash'],
    [3008, 'Sint', 'Sont'],
    [3009, 'Sigma', 'Sigmoid'],
    [3010, 'Stan', 'Stanislav']]


buyer_keys = ['id', 'first_name', 'last_name']
buyer_values = [
    [4001, 'Barbara', 'Bento'],
    [4002, 'Boyu', 'Bland'],
    [4003, 'Brendon', 'Bright'],
    [4004, 'Breston', 'Brush'],
    [4005, 'Brinta', 'Broy'],
    [4006, 'Blent', 'Buq'],
    [4007, 'Boppi', 'Boppal'],
    [4008, 'Ben', 'Bench'],
    [4009, 'Barbie', 'Barb'],
    [4010, 'Bovi', 'Bentie'],
    [4011, 'Bull', 'Bully'],
    [4012, 'Binnie', 'Bun']]


#--------------------------------------------------
# Create house listings and sales
#--------------------------------------------------

house_keys = ['id', 'name', 'office_id', 'agent_id', 'seller_id', 'bedrooms', 'bathrooms', 'listing_price',
             'zipcode', 'listing_date', 'sold']
house_values = [
    [5001, 'Magnificent Mansion', 1001, 2001, 3001, 10, 8, 3000000, 10911, datetime.date(2019, 1, 1), False],
    [5002, 'Handsome House', 1001, 2001, 3001, 4, 2, 120000, 10353, datetime.date(2019, 1, 3), False],
    [5003, 'Comfy Cabin', 1002, 2001, 3002, 2, 1, 90000, 24502, datetime.date(2019, 1, 4), False],
    [5004, 'Awesome Apartment', 1002, 2002, 3003, 4, 2, 600000, 24502, datetime.date(2019, 1, 6), False],
    [5005, 'Random Room', 1003, 2002, 3003, 1, 1, 300000, 33333, datetime.date(2019, 1, 8), False],
    [5006, 'Buggy Bed', 1003, 2003, 3002, 1, 1, 14000, 34519, datetime.date(2019, 1, 12), False],
    [5007, 'Vengeful Villa', 1003, 2003, 3001, 8, 4, 56000, 32025, datetime.date(2019, 1, 19), False],
    [5008, 'Full Flat', 1004, 2003, 3001, 4, 2, 400000, 33333, datetime.date(2019, 1, 20), False],
    [5009, 'Pizza Palace', 1004, 2003, 3001, 12, 3, 5000000, 32025, datetime.date(2019, 1, 20), False],
    [5010, 'Superhigh Skyscraper', 1004, 2003, 3001, 5, 5, 600000, 24502, datetime.date(2019, 1, 22), False],
    [5011, 'Risky Rooftop', 1004, 2003, 3001, 2, 1, 40000, 10911, datetime.date(2019, 1, 23), False],
    [5012, 'Shiny Studio', 1005, 2003, 3001, 3, 1, 29000, 32025, datetime.date(2019, 1, 29), False],
    [5013, 'Crunchy Cellar', 1005, 2003, 3001, 1, 0, 500, 24502, datetime.date(2019, 1, 31), False],
    [5014, 'Magnificent Mansion', 1006, 2003, 3003, 6, 3, 79000, 24502, datetime.date(2019, 1, 31), False]]


In [4]:
#--------------------------------------------------
# Add entries to session
#--------------------------------------------------


keys = [office_keys, agent_keys, seller_keys, buyer_keys, house_keys]
values = [office_values, agent_values, seller_values, buyer_values, house_values]
table_classes = [Office, Agent, Seller, Buyer, House]


def add_entries_to_session(keys, values, table_class):
    list_of_dict = []
    for value in values:
        item_dict = dict(zip(keys, value))
        list_of_dict.append(item_dict)
    
    for data_entry in list_of_dict:
        entry = table_class(**data_entry)
        session.add(entry)

        
for i in range(len(table_classes)):
    add_entries_to_session(keys[i], values[i], table_classes[i])

session.commit()
session.close()

### Show tables

**Office**

In [5]:
pd.read_sql(session.query(Office).statement, session.bind)

,id,location
0,1001,San Francisco
1,1002,Seoul
2,1003,Hyderabad
3,1004,Berlin
4,1005,Buenos Aires
5,1006,London
6,1007,Taipei


**Agent**

In [6]:
pd.read_sql(session.query(Agent).statement, session.bind)

,id,first_name,last_name,email
0,2001,Anna,Aruya,anna@google.com
1,2002,Anton,Aldi,anton@google.com
2,2003,Ashish,Alder,ashish@google.com
3,2004,Angela,Anger,angela@google.com
4,2005,Adrian,Aldi,adrian@google.com
5,2006,Ari,Aro,ari@google.com
6,2007,Anne,Arido,anne@google.com
7,2008,Alinga,Azon,alinga@google.com
8,2009,Abuli,Appo,abuli@google.com
9,2010,Ankita,Avush,ankita@google.com


**Seller**

In [7]:
pd.read_sql(session.query(Seller).statement, session.bind)

,id,first_name,last_name
0,3001,Sabine,Singer
1,3002,Sergio,Sent
2,3003,Sasha,Sorrow
3,3004,Stub,Stringo
4,3005,Stella,Serg
5,3006,Sanik,Soumka
6,3007,Soumaya,Strash
7,3008,Sint,Sont
8,3009,Sigma,Sigmoid
9,3010,Stan,Stanislav


**Buyer**

In [8]:
pd.read_sql(session.query(Buyer).statement, session.bind)

,id,first_name,last_name
0,4001,Barbara,Bento
1,4002,Boyu,Bland
2,4003,Brendon,Bright
3,4004,Breston,Brush
4,4005,Brinta,Broy
5,4006,Blent,Buq
6,4007,Boppi,Boppal
7,4008,Ben,Bench
8,4009,Barbie,Barb
9,4010,Bovi,Bentie


**House**

In [9]:
pd.read_sql(session.query(House).statement, session.bind)

,id,name,office_id,agent_id,seller_id,bedrooms,bathrooms,listing_price,zipcode,listing_date,sold
0,5001,Magnificent Mansion,1001,2001,3001,10,8,3000000,10911,2019-01-01,False
1,5002,Handsome House,1001,2001,3001,4,2,120000,10353,2019-01-03,False
2,5003,Comfy Cabin,1002,2001,3002,2,1,90000,24502,2019-01-04,False
3,5004,Awesome Apartment,1002,2002,3003,4,2,600000,24502,2019-01-06,False
4,5005,Random Room,1003,2002,3003,1,1,300000,33333,2019-01-08,False
5,5006,Buggy Bed,1003,2003,3002,1,1,14000,34519,2019-01-12,False
6,5007,Vengeful Villa,1003,2003,3001,8,4,56000,32025,2019-01-19,False
7,5008,Full Flat,1004,2003,3001,4,2,400000,33333,2019-01-20,False
8,5009,Pizza Palace,1004,2003,3001,12,3,5000000,32025,2019-01-20,False
9,5010,Superhigh Skyscraper,1004,2003,3001,5,5,600000,24502,2019-01-22,False


### Add additional data

In [10]:
def house_sale(house_id, buyer_id, agent_id, sale_date):
    '''Adds a sale to a transaction and commits it to the database. '''

    # Find initial listing price
    listing_price = session.query(House.listing_price).filter(House.id == house_id).first()[0]
    
    # Add agent's commission
    bracket = case([(House.listing_price < 100000, 0.01),
           (House.listing_price < 200000, 0.075),
           (House.listing_price < 500000, 0.06),
           (House.listing_price < 1000000, 0.05),
           (House.listing_price > 1000000, 0.04),])
    agent_commission = session.query(House.listing_price*bracket).filter(House.id == 5001).first()[0]
    
    # Calculate sale price
    sale_price = listing_price + agent_commission
    
    # Add Sale entry (id autoincrements)
    sale_entry = Sale( 
        house_id = house_id,
        buyer_id = buyer_id,
        sale_price = sale_price,
        sale_date = sale_date)
    session.add(sale_entry)
    
    
    # Mark house as sold in House table
    house_sold = session.query(House).filter(House.id == house_id)
    house_sold.update({House.sold: True})

    
    # Add commission entry (id autoincrements)
    sale_id = session.query(Sale.id).filter(House.id == house_id).first()[0]
    
    commission_entry = Commission(
        agent_id = agent_id,
        sale_id = sale_id,
        commission = agent_commission)
    session.add(commission_entry)

    # Commit the transaction
    session.commit()
    

In [11]:
# Add house sales (house_id, buyer_id, agent_id, sale_date))

house_sale(5001, 4001, 2001, datetime.date(2019, 4, 1))
house_sale(5002, 4002, 2001, datetime.date(2019, 5, 1))
house_sale(5003, 4003, 2001, datetime.date(2019, 5, 1))
house_sale(5004, 4004, 2002, datetime.date(2019, 6, 1))
house_sale(5005, 4005, 2002, datetime.date(2019, 4, 1))
house_sale(5006, 4006, 2003, datetime.date(2019, 4, 1))
house_sale(5007, 4007, 2003, datetime.date(2019, 7, 1))
house_sale(5008, 4007, 2004, datetime.date(2019, 7, 1))
house_sale(5009, 4008, 2004, datetime.date(2019, 8, 1))
house_sale(5010, 4008, 2005, datetime.date(2019, 8, 1))

In [12]:
# Sale Data
pd.read_sql(session.query(Sale).statement, session.bind)

,id,house_id,buyer_id,sale_price,sale_date
0,1,5001,4001,3120000,2019-04-01
1,2,5002,4002,240000,2019-05-01
2,3,5003,4003,210000,2019-05-01
3,4,5004,4004,720000,2019-06-01
4,5,5005,4005,420000,2019-04-01
5,6,5006,4006,134000,2019-04-01
6,7,5007,4007,176000,2019-07-01
7,8,5008,4007,520000,2019-07-01
8,9,5009,4008,5120000,2019-08-01
9,10,5010,4008,720000,2019-08-01


In [13]:
# Updated sold status to 'True'
pd.read_sql(session.query(House).statement, session.bind)

,id,name,office_id,agent_id,seller_id,bedrooms,bathrooms,listing_price,zipcode,listing_date,sold
0,5001,Magnificent Mansion,1001,2001,3001,10,8,3000000,10911,2019-01-01,True
1,5002,Handsome House,1001,2001,3001,4,2,120000,10353,2019-01-03,True
2,5003,Comfy Cabin,1002,2001,3002,2,1,90000,24502,2019-01-04,True
3,5004,Awesome Apartment,1002,2002,3003,4,2,600000,24502,2019-01-06,True
4,5005,Random Room,1003,2002,3003,1,1,300000,33333,2019-01-08,True
5,5006,Buggy Bed,1003,2003,3002,1,1,14000,34519,2019-01-12,True
6,5007,Vengeful Villa,1003,2003,3001,8,4,56000,32025,2019-01-19,True
7,5008,Full Flat,1004,2003,3001,4,2,400000,33333,2019-01-20,True
8,5009,Pizza Palace,1004,2003,3001,12,3,5000000,32025,2019-01-20,True
9,5010,Superhigh Skyscraper,1004,2003,3001,5,5,600000,24502,2019-01-22,True


# Queries

### 1. Top 5 offices with most sales

In [14]:
stmt = session.query(
    House.office_id,
    Office.location,
    func.count(House.office_id)
     ).join(Office).filter(House.sold == True).group_by(House.office_id).limit(5)

pd.read_sql(stmt.statement, session.bind)


,office_id,location,count_1
0,1001,San Francisco,2
1,1002,Seoul,2
2,1003,Hyderabad,3
3,1004,Berlin,3


### 2. Top 5 estate agents who sold the most (contact details and sales details)

In [15]:
stmt = session.query(
    Agent.id,
    Agent.first_name,
    Agent.last_name,
    Agent.email,
    func.count(House.agent_id)
     ).join(House).filter(House.sold == True).group_by(Agent.id).limit(5)

pd.read_sql(stmt.statement, session.bind)


,id,first_name,last_name,email,count_1
0,2001,Anna,Aruya,anna@google.com,3
1,2002,Anton,Aldi,anton@google.com,2
2,2003,Ashish,Alder,ashish@google.com,5


### 3. Commission of each estate agent

In [16]:
# Creates a summary table of output 

summary = session.query(
    Agent.first_name, 
    Agent.last_name,
    Commission.agent_id, 
    func.count(Commission.agent_id),
    func.sum(Sale.sale_price),
    func.sum(Commission.commission)).group_by(Agent.id). \
        join(Commission).join(Sale).statement

pd.read_sql(summary, session.bind)

,first_name,last_name,agent_id,count_1,sum_1,sum_2
0,Anna,Aruya,2001,3,9360000,360000
1,Anton,Aldi,2002,2,6240000,240000
2,Ashish,Alder,2003,2,6240000,240000
3,Angela,Anger,2004,2,6240000,240000
4,Adrian,Aldi,2005,1,3120000,120000


### 4. Average number of days a house was on the market
Unfortunately, I did not manage to find out how to get at the average number of days. I researched a lot, but the combination of SQLAlchemy + Sqllite seems to be a bit tricky. Below are different versions I tried, and here is a conversation around the problem: https://stackoverflow.com/questions/1385393/how-to-get-sqlalchemy-storing-datetime-as-julianday-in-sqlite?rq=1

In [17]:
# HOW I USE DATES: house_sale(5010, 4008, 2005, datetime.date(2019, 8, 1))

stmt = session.query(
    House.listing_date,
    Sale.sale_date,
    # func.datediff(text('month'), Sale.sale_date, House.listing_date)
    # func.trunc((extract('epoch', Sale.sale_date) - extract('epoch', House.listing_date)) / 60)
    # func.datediff('month', Sale.sale_date, House.listing_date)
    func.julianday(Sale.sale_date, House.listing_date)
     ).join(Sale)

pd.read_sql(stmt.statement, session.bind)


,listing_date,sale_date,julianday_1
0,2019-01-01,2019-04-01,None
1,2019-01-03,2019-05-01,None
2,2019-01-04,2019-05-01,None
3,2019-01-06,2019-06-01,None
4,2019-01-08,2019-04-01,None
5,2019-01-12,2019-04-01,None
6,2019-01-19,2019-07-01,None
7,2019-01-20,2019-07-01,None
8,2019-01-20,2019-08-01,None
9,2019-01-22,2019-08-01,None


### 5. Average selling price of a house

In [18]:
stmt = session.query(
    func.avg(Sale.sale_price)
     ).one()[0]

print('Average selling price:', stmt, 'USD')

Average selling price: 1138000.0 USD


### 6. Zip codes with top 5 average selling price

In [19]:
# extend: order by 

stmt = session.query(
    Office.location,
    House.zipcode,
    func.avg(Sale.sale_price)
     ).join(House).join(Sale). \
    filter(House.sold == True).group_by(House.zipcode).limit(5)

pd.read_sql(stmt.statement, session.bind)


,location,zipcode,avg_1
0,San Francisco,10353,240000.0
1,San Francisco,10911,3120000.0
2,Berlin,24502,550000.0
3,Berlin,32025,2648000.0
4,Berlin,33333,470000.0


### Cleanup

In [20]:
session.close()
Base.metadata.drop_all(bind=engine) 